# 初めてのインテリジェントエージェントチームを構築する：ADK を使用した段階的な旅行コンシェルジュボット


## 事前準備

In [ ]:
# @title 必要な依存ライブラリをインストール
%pip install "google-cloud-aiplatform[agent_engines,adk]==1.96.0" -q
%pip install google-adk==1.2.1  -q
%pip install google-genai==1.20.0 -q

In [ ]:
# 以下のコマンドをターミナルで実行してください
gcloud auth application-default login

In [4]:
# @title APIキーの設定
import os
import vertexai

# Vertex AI API　使用設定
GOOGLE_CLOUD_PROJECT = "agent-dev-workshop" # @param {type:"string"}
STAGING_BUCKET = "gs://tayzar-bucket" #@param {type:"string"}
LOCATION = "us-central1"

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "True"
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_CLOUD_PROJECT"] = GOOGLE_CLOUD_PROJECT

vertexai.init(
    project=GOOGLE_CLOUD_PROJECT,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

# --- 使いやすくするためにモデル定数を定義します ---
DEFAULT_MODEL = "gemini-2.5-flash"

!gcloud auth application-default set-quota-project $GOOGLE_CLOUD_PROJECT

import logging

class _NoFunctionCallWarning(logging.Filter):
    def filter(self, record: logging.LogRecord) -> bool:
        message = record.getMessage()
        if "there are non-text parts in the response:" in message:
            return False
        else:
            return True

logging.getLogger("google_genai.types").addFilter(_NoFunctionCallWarning())


Credentials saved to file: [/Users/tayzar/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "agent-dev-workshop" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


---

## 第１章：初めてのエージェント - 基本的な旅行情報検索


###  1\. Toolの定義 (get_destination_info)

**docstring は重要です！**
エージェントの LLM は、関数の docstring から以下の情報を取得します。
*  ツールが何をするか。
*  いつ使用するか。
*  どの引数が必要か（destination: str）。
*  どの情報を返すか。

In [5]:
# 旅行先情報取得関数
def get_destination_info(destination: str) -> dict:
    """指定された旅行先の基本情報を取得します。

    Args:
        destination (str): 旅行先の名前（例：「ニューヨーク」、「ロンドン」、「東京」）。

    Returns:
        dict: 旅行先情報を含む辞書。
              'status' キー（'success' または 'error'）を含みます。
              'success' の場合、旅行先の詳細情報を持つ 'info' キーを含みます。
              'error' の場合、'error_message' キーを含みます。
    """

    mock_destination_db = {
        "ニューヨーク": {"status": "success", "info": "ニューヨークはアメリカ合衆国最大の都市で、自由の女神像、セントラルパーク、タイムズスクエアなどの観光名所があります。"},
        "ロンドン": {"status": "success", "info": "ロンドンはイギリスの首都で、バッキンガム宮殿、ビッグベン、ロンドン塔などの歴史的建造物が有名です。"},
        "東京": {"status": "success", "info": "東京は日本の首都で、伝統と現代が融合した都市です。浅草寺、東京スカイツリー、渋谷などが人気の観光スポットです。"},
    }

    if destination in mock_destination_db:
        return mock_destination_db[destination]
    else:
        return {"status": "error", "error_message": f"申し訳ありませんが、「{destination}」の情報はありません。"}

print(get_destination_info("ニューヨーク"))
print(get_destination_info("パリ"))

{'status': 'success', 'info': 'ニューヨークはアメリカ合衆国最大の都市で、自由の女神像、セントラルパーク、タイムズスクエアなどの観光名所があります。'}
{'status': 'error', 'error_message': '申し訳ありませんが、「パリ」の情報はありません。'}


### 2\. エージェントの定義 (`travel_agent`)

In [6]:
from google.adk.agents import Agent

# LLM Agentオブジェクト初期化
travel_agent = Agent(
    name="travel_agent_v1",
    model=DEFAULT_MODEL,
    description="特定の旅行先の情報を提供します。",
    instruction="あなたは親切な旅行アシスタントです。旅行コンシェルジュのように回答してください",
    tools=[get_destination_info],
)

### 3\.Local Agent Wrapper

In [7]:
import datetime, time, uuid
from google.genai.types import Part, Content
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

# ADKの主要オブジェクト三兄弟のクラス
class LocalAgent:
    def __init__(self, agent, app_name='default_app', user_id='default_user', debug=False, initial_state = None):
        self._agent = agent
        self._app_name = app_name
        self._user_id = user_id
        self._runner = Runner(
            app_name=self._app_name,
            agent=self._agent,
            session_service=InMemorySessionService()
        )
        self._session = None
        self._debug = debug
        self._initial_state = initial_state

    async def stream(self, query):
        if not self._session:
            self._session = await self._runner.session_service.create_session(
                app_name=self._app_name,
                user_id=self._user_id,
                session_id=uuid.uuid4().hex,
                state = self._initial_state
            )
        content = Content(role='user', parts=[Part.from_text(text=query)])
        async_events = self._runner.run_async(
            user_id=self._user_id,
            session_id=self._session.id,
            new_message=content,
        )
        result = []

        async for event in async_events:
            if self._debug:
                print(f"呼び出しAgent: {event.author}")
            if (event.content and event.content.parts):

                response = '\n'.join([p.text for p in event.content.parts if p.text])
                if self._debug:
                    function_call = '\n'.join([p.function_call.name for p in event.content.parts if p.function_call])
                    if function_call:
                        print(f"呼び出しTool: {function_call}")
                if response:
                    print(response)
                    result.append(response)
        return result

### 4\. テストしてみましょう！


In [8]:
client = LocalAgent(travel_agent)
print("------------------message (1)------------------")
_ = await client.stream("東京の観光情報を教えてください")

print("\n------------------message (2)------------------")
_ = await client.stream("パリはどうですか？")

print("\n------------------message (3)------------------")
_ = await client.stream("ニューヨークについて教えてください")


------------------message (1)------------------
東京は日本の首都で、伝統と現代が融合した都市です。浅草寺、東京スカイツリー、渋谷などが人気の観光スポットです。

------------------message (2)------------------
申し訳ありませんが、「パリ」の情報はありません。

------------------message (3)------------------
ニューヨークはアメリカ合衆国最大の都市で、自由の女神像、セントラルパーク、タイムズスクエアなどの観光名所があります。


---

おめでとうございます！最初の ADK エージェントの構築と対話に成功しました。エージェントはユーザーのリクエストを理解し、ツールを使用して情報を見つけ、ツールの結果に基づいて適切に応答します。
次のステップでは、このエージェントを強化する基盤となる言語モデルを簡単に切り替える方法を探ります。


## 第２章：エージェントチームの構築 - 予約、インスピレーション、プランニングの委任


### 1\. サブエージェント用のツールを定義する


In [10]:
# 予約確認ツール
def create_reservation(booking_details: str) -> dict:
    """予約を作成し、確認情報を返します。

    Args:
        booking_details (str): 予約の詳細情報（例：ホテル名、日付、人数など）

    Returns:
        dict: 予約確認情報を含む辞書
    """
    print(f"--- ツール: create_reservation が呼び出されました。詳細: {booking_details} ---")
    return {
        "status": "success",
        "confirmation_code": "BK12345",
        "details": f"予約が確認されました: {booking_details}",
        "timestamp": "2023-12-01T10:00:00Z"
    }

# 支払い方法選択ツール
def payment_options() -> dict:
    """利用可能な支払い方法のリストを返します。

    Returns:
        dict: 利用可能な支払い方法のリスト
    """
    print(f"--- ツール: payment_options が呼び出されました ---")
    return {
        "status": "success",
        "options": [
            {"id": "cc", "name": "クレジットカード", "fee": "0円"},
            {"id": "bank", "name": "銀行振込", "fee": "330円"},
            {"id": "pay_app", "name": "モバイル決済", "fee": "0円"}
        ]
    }

# 支払い処理ツール
def process_payment(payment_method: str, amount: float) -> dict:
    """支払いを処理し、結果を返します。

    Args:
        payment_method (str): 支払い方法のID
        amount (float): 支払い金額

    Returns:
        dict: 支払い処理の結果
    """
    print(f"--- ツール: process_payment が呼び出されました。方法: {payment_method}, 金額: {amount} ---")
    return {
        "status": "success",
        "transaction_id": "TX98765",
        "payment_method": payment_method,
        "amount": amount,
        "timestamp": "2023-12-01T10:15:00Z"
    }

# 旅行先提案ツール
def suggest_destinations(preferences: str) -> dict:
    """ユーザーの好みに基づいて旅行先を提案します。

    Args:
        preferences (str): ユーザーの旅行の好み（例：「ビーチ、リラックス」「都市、文化」など）

    Returns:
        dict: 提案された旅行先のリスト
    """
    print(f"--- ツール: suggest_destinations が呼び出されました。好み: {preferences} ---")

    destinations = {
        "ビーチ": ["沖縄", "ハワイ", "バリ島"],
        "都市": ["東京", "ニューヨーク", "パリ"],
        "自然": ["北海道", "スイス", "ニュージーランド"],
        "文化": ["京都", "ローマ", "エジプト"]
    }

    # 好みのキーワードに基づいて行き先を選択
    results = []
    for key, places in destinations.items():
        if key in preferences:
            results.extend(places)

    # 結果がない場合はデフォルトの提案
    if not results:
        results = ["東京", "京都", "沖縄", "北海道", "ハワイ"]

    return {
        "status": "success",
        "destinations": results,
        "based_on": preferences
    }

# 観光スポット提案ツール
def suggest_attractions(destination: str) -> dict:
    """指定された旅行先の観光スポットを提案します。

    Args:
        destination (str): 旅行先の名前

    Returns:
        dict: 提案された観光スポットのリスト
    """
    print(f"--- ツール: suggest_attractions が呼び出されました。旅行先: {destination} ---")

    attractions = {
        "東京": ["東京スカイツリー", "浅草寺", "渋谷スクランブル交差点", "上野公園", "東京ディズニーランド"],
        "京都": ["金閣寺", "伏見稲荷大社", "清水寺", "嵐山", "二条城"],
        "沖縄": ["美ら海水族館", "首里城", "古宇利島", "万座毛", "竹富島"],
        "ニューヨーク": ["自由の女神", "セントラルパーク", "タイムズスクエア", "エンパイアステートビル", "ブルックリン橋"],
        "パリ": ["エッフェル塔", "ルーブル美術館", "ノートルダム大聖堂", "凱旋門", "モンマルトル"]
    }

    # 指定された旅行先の観光スポットを返す
    if destination in attractions:
        return {
            "status": "success",
            "attractions": attractions[destination],
            "destination": destination
        }
    else:
        return {
            "status": "error",
            "message": f"{destination}の観光スポット情報は利用できません。"
        }

# 旅程作成ツール
def create_itinerary(destination: str, days: int) -> dict:
    """指定された旅行先と日数に基づいて旅程を作成します。

    Args:
        destination (str): 旅行先の名前
        days (int): 旅行日数

    Returns:
        dict: 作成された旅程
    """
    print(f"--- ツール: create_itinerary が呼び出されました。旅行先: {destination}, 日数: {days} ---")

    # 簡易的な旅程テンプレート
    itinerary = {
        "destination": destination,
        "days": days,
        "schedule": []
    }

    # 日数分の簡易スケジュールを生成
    for day in range(1, days + 1):
        daily_plan = {
            "day": day,
            "morning": f"{destination}の観光スポット訪問",
            "afternoon": "地元料理を楽しむランチ",
            "evening": "ホテルでリラックス"
        }
        itinerary["schedule"].append(daily_plan)

    return {
        "status": "success",
        "itinerary": itinerary
    }

# ホテル検索ツール
def search_hotels(destination: str, check_in: str, check_out: str) -> dict:
    """指定された条件でホテルを検索します。

    Args:
        destination (str): 旅行先の名前
        check_in (str): チェックイン日（YYYY-MM-DD形式）
        check_out (str): チェックアウト日（YYYY-MM-DD形式）

    Returns:
        dict: 検索結果のホテルリスト
    """
    print(f"--- ツール: search_hotels が呼び出されました。旅行先: {destination}, チェックイン: {check_in}, チェックアウト: {check_out} ---")

    # モックのホテルデータ
    hotels = [
        {"name": f"{destination}グランドホテル", "rating": 4.5, "price": 25000, "amenities": ["プール", "スパ", "レストラン"]},
        {"name": f"{destination}ビジネスホテル", "rating": 3.8, "price": 12000, "amenities": ["Wi-Fi", "朝食"]},
        {"name": f"{destination}リゾート", "rating": 4.7, "price": 35000, "amenities": ["ビーチアクセス", "プール", "スパ", "複数レストラン"]}
    ]

    return {
        "status": "success",
        "hotels": hotels,
        "destination": destination,
        "check_in": check_in,
        "check_out": check_out
    }

# フライト検索ツール
def search_flights(origin: str, destination: str, date: str) -> dict:
    """指定された条件でフライトを検索します。

    Args:
        origin (str): 出発地
        destination (str): 目的地
        date (str): 出発日（YYYY-MM-DD形式）

    Returns:
        dict: 検索結果のフライトリスト
    """
    print(f"--- ツール: search_flights が呼び出されました。出発地: {origin}, 目的地: {destination}, 日付: {date} ---")

    # モックのフライトデータ
    flights = [
        {"airline": "日本航空", "flight_number": "JL123", "departure": "09:00", "arrival": "11:30", "price": 45000},
        {"airline": "全日空", "flight_number": "NH456", "departure": "13:00", "arrival": "15:30", "price": 42000},
        {"airline": "LCCエアライン", "flight_number": "LC789", "departure": "07:00", "arrival": "09:30", "price": 28000}
    ]

    return {
        "status": "success",
        "flights": flights,
        "origin": origin,
        "destination": destination,
        "date": date
    }


### 2\.  サブエージェント（予約、インスピレーション、プランニング）を定義する


In [11]:
from google.genai.types import GenerateContentConfig


booking_agent = Agent(
    model=DEFAULT_MODEL,
    name="booking_agent",
    description="予約の確認と支払い処理を行うエージェント",
    instruction="""あなたは旅行予約エージェントです。
    ユーザーの予約確認と支払い処理を担当します。
    予約を確認するには 'create_reservation' ツールを使用してください。
    支払い方法を表示するには 'payment_options' ツールを使用してください。
    支払いを処理するには 'process_payment' ツールを使用してください。
    常に丁寧かつプロフェッショナルな対応を心がけてください。""",
    tools=[create_reservation, payment_options, process_payment],
    generate_content_config=GenerateContentConfig(temperature=0.2)
)


inspiration_agent = Agent(
    model=DEFAULT_MODEL,
    name="inspiration_agent",
    description="旅行先のアイデアや観光スポットを提案するエージェント",
    instruction="""あなたは旅行インスピレーションエージェントです。
    ユーザーの好みや興味に基づいて旅行先を提案します。
    旅行先を提案するには 'suggest_destinations' ツールを使用してください。
    観光スポットを提案するには 'suggest_attractions' ツールを使用してください。
    常に熱意を持って旅行の魅力を伝えてください。""",
    tools=[suggest_destinations, suggest_attractions],
    generate_content_config=GenerateContentConfig(temperature=0.5)
)


planning_agent = Agent(
    model=DEFAULT_MODEL,
    name="planning_agent",
    description="旅行の計画と手配を行うエージェント",
    instruction="""あなたは旅行計画エージェントです。
    ユーザーの旅行計画を立て、ホテルやフライトの検索を行います。
    旅程を作成するには 'create_itinerary' ツールを使用してください。
    ホテルを検索するには 'search_hotels' ツールを使用してください。
    フライトを検索するには 'search_flights' ツールを使用してください。
    常に効率的で実用的な旅行計画を提案してください。""",
    tools=[create_itinerary, search_hotels, search_flights],
    generate_content_config=GenerateContentConfig(temperature=0.3)
)


### 3\.  サブエージェントを持つルートエージェントを定義する（Travel Agent v2）



In [12]:
travel_agent_team = Agent(
    name="travel_agent_v2", # 新しいバージョン名を付ける
    model=DEFAULT_MODEL,
    description="メインコーディネーターエージェント。旅行リクエストを処理し、予約、インスピレーション、プランニングを専門家に委任します。",
    instruction=""""
                あなたはチームを調整するメイン旅行エージェントです。あなたの主な責任は旅行情報を提供することです。
                旅行のインスピレーションが必要な場合は 'inspiration_agent' に、
                旅行の計画が必要な場合は 'planning_agent' に、
                予約の処理が必要な場合は 'booking_agent' に委任してください。
                """,
    tools=[get_destination_info],
    sub_agents=[booking_agent, inspiration_agent, planning_agent]
)



### 4\. テストしてみましょう！

以下のテストでは、マルチエージェントシステムがどのように連携して動作するかを確認します。特に注目すべき点は：

1. **委任の仕組み**: ルートエージェント（travel_agent_team）は、リクエストの内容に応じて適切なサブエージェントにタスクを委任します。
2. **専門性の活用**: 各サブエージェントは特定の領域に特化しており、それぞれが専門的なツールを使用します。
   - inspiration_agent: 旅行のアイデアや観光スポットの提案
   - planning_agent: 旅行計画の作成、ホテルやフライトの検索
   - booking_agent: 予約の確認と支払い処理
3. **ツールの使用**: 各エージェントは、割り当てられたタスクを実行するために適切なツールを選択します。

デバッグモードを有効にしているため、各リクエストに対してどのエージェントが呼び出され、どのツールが使用されるかを確認できます。


In [14]:

client = LocalAgent(travel_agent_team, debug=True)

print("------------------テスト 1: 基本的な旅行情報リクエスト------------------")
"""
ルートエージェントが基本的な旅行情報リクエストを処理する様子を確認します。
ルートエージェントは自身の get_destination_info ツールを使用して情報を提供します
"""
_ = await client.stream("東京について教えてください")

print("\n------------------テスト 2: 旅行インスピレーション (inspiration_agent への委任)------------------")
"""
ルートエージェントが旅行のアイデアに関するリクエストを inspiration_agent に委任する様子を確認します。
inspiration_agent は suggest_destinations ツールを使用して、ユーザーの好みに合った旅行先を提案します。
"""
_ = await client.stream("ビーチでリラックスできる旅行先を提案してください")

print("\n------------------テスト 3: 観光スポット情報 (inspiration_agent への委任)------------------")
"""
このテストでは、ルートエージェントが観光スポット情報のリクエストを inspiration_agent に委任する様子を確認します
inspiration_agent は suggest_attractions ツールを使用して、指定された旅行先の観光スポットを提案します
"""
_ = await client.stream("京都のおすすめ観光スポットを教えてください")

print("\n------------------テスト 4: 旅行計画 (planning_agent への委任)------------------")
"""
このテストでは、ルートエージェントが旅行計画のリクエストを planning_agent に委任する様子を確認します。
planning_agent は create_itinerary ツールを使用して、旅行の日程を作成します。
"""
_ = await client.stream("東京への3日間の旅行プランを作成してください")

print("\n------------------テスト 5: ホテル検索 (planning_agent への委任)------------------")
"""
このテストでは、ルートエージェントがホテル検索のリクエストを planning_agent に委任する様子を確認します
planning_agent は search_hotels ツールを使用して、指定された条件でホテルを検索します。
"""
_ = await client.stream("東京で2023年12月25日から12月28日まで宿泊できるホテルを探してください")

print("\n------------------テスト 6: 予約確認 (booking_agent への委任)------------------")
"""
このテストでは、ルートエージェントが予約確認のリクエストを booking_agent に委任する様子を確認します
booking_agent は create_reservation ツールを使用して、予約を確認します
"""
_ = await client.stream("東京グランドホテルの予約を確認してください")

print("\n------------------テスト 7: 支払い処理 (booking_agent への委任)------------------")
"""このテストでは、ルートエージェントが支払い処理のリクエストを booking_agent に委任する様子を確認します。
booking_agent は payment_options と process_payment ツールを使用して、支払い方法の表示と処理を行います
"""
_ = await client.stream("予約の支払い方法を教えてください。クレジットカードで35000円を支払いたいです")

print("\n------------------マルチエージェント委任のまとめ------------------")
print("上記のテストを通じて、以下のことが確認できました：")
print("1. ルートエージェントは、リクエストの内容を理解し、適切なサブエージェントに委任します。")
print("2. 各サブエージェントは、自分の専門領域に関するリクエストを処理します：")
print("   - inspiration_agent: 旅行先の提案や観光スポット情報の提供")
print("   - planning_agent: 旅行計画の作成、ホテルやフライトの検索")
print("   - booking_agent: 予約の確認と支払い処理")
print("3. 各エージェントは、タスクを実行するために適切なツールを選択します。")
print("4. デバッグ出力から、どのエージェントがどのツールを使用したかを確認できます。")
print("\nこのようなマルチエージェントアーキテクチャにより、複雑なタスクを専門的なサブエージェントに分割し、")
print("効率的かつ効果的に処理することができます。")

------------------テスト 1: 基本的な旅行情報リクエスト------------------
呼び出しAgent: travel_agent_v2
呼び出しTool: get_destination_info
呼び出しAgent: travel_agent_v2
呼び出しAgent: travel_agent_v2
東京は日本の首都で、伝統と現代が融合した都市です。浅草寺、東京スカイツリー、渋谷などが人気の観光スポットです。

------------------テスト 2: 旅行インスピレーション (inspiration_agent への委任)------------------
呼び出しAgent: travel_agent_v2
呼び出しTool: transfer_to_agent
呼び出しAgent: travel_agent_v2
呼び出しAgent: inspiration_agent
呼び出しTool: suggest_destinations
--- ツール: suggest_destinations が呼び出されました。好み: ビーチ、リラックス ---
呼び出しAgent: inspiration_agent
呼び出しAgent: inspiration_agent
ビーチでリラックスできる最高の旅行先をいくつかご提案できます！

*   **沖縄**: 日本にいながらにして美しいビーチと独特の文化を体験できる、まさに楽園です。透き通った海と白い砂浜があなたを待っています！
*   **ハワイ**: 言わずと知れたビーチリゾートの王道！サーフィン、シュノーケリング、そして息をのむような夕日…最高の思い出が作れること間違いなしです！
*   **バリ島**: 神々の島として知られるバリ島は、美しいビーチだけでなく、豊かな文化とスピリチュアルな体験もできる場所です。心身ともにリフレッシュできるでしょう！

これらの素晴らしい場所で、日頃の疲れを癒し、最高のビーチ体験を満喫してください！

------------------テスト 3: 観光スポット情報 (inspiration_agent への委任)------------------
呼び出しAgent: inspiration_agent
呼び出しTool: sugge

## 第３章：Session Stateでメモリとパーソナライゼーションを追加する


### 1\.  状態を意識した旅行情報ツールを作成する (`get_destination_info_stateful`)


In [ ]:
from google.adk.tools.tool_context import ToolContext
from datetime import datetime
from google.adk.agents.callback_context import CallbackContext

# 旅行に関する定数
SYSTEM_TIME = "_time"
ITIN_INITIALIZED = "_itin_initialized"
ITIN_KEY = "itinerary"
PROF_KEY = "user_profile"
ITIN_START_DATE = "itinerary_start_date"
ITIN_END_DATE = "itinerary_end_date"
ITIN_DATETIME = "itinerary_datetime"
START_DATE = "start_date"
END_DATE = "end_date"
USER_PREFERENCES = "user_preferences"

# Session Stateに基づいて旅行情報を返す関数
def get_destination_info_stateful(destination: str, tool_context: ToolContext) -> dict:
    """指定された旅行先の情報を取得します、セッションの状態に基づいて言語を変換します。

    Args:
        destination (str): 旅行先の名前（例：「ニューヨーク」、「ロンドン」、「東京」）。
        tool_context (ToolContext): ツール呼び出しのコンテキストを提供し、呼び出しコンテキスト、関数呼び出しID、イベントアクション、認証レスポンスへのアクセスを含みます。

    Returns:
        dict: 旅行先情報を含む辞書。
              'status' キー（'success' または 'error'）を含みます。
              'success' の場合、旅行先の詳細情報を持つ 'info' キーを含みます。
              'error' の場合、'error_message' キーを含みます。
    """

    print(f"--- ツール: get_destination_info_stateful が {destination} のために呼び出されました ---")

    # --- 状態から設定を読み込み ---
    user_preferences = tool_context.state.get(USER_PREFERENCES, {})
    preferred_language = user_preferences.get("language", "Japanese")
    print(f"--- ツール: 状態 'user_preferences.language' を読み込み中: {preferred_language} ---")

    # 旅行の日程情報を取得
    itinerary = tool_context.state.get(ITIN_KEY, {})
    current_time = tool_context.state.get(SYSTEM_TIME, str(datetime.now()))

    # 旅行の日程情報をログに出力
    if itinerary:
        start_date = itinerary.get(START_DATE, "未設定")
        end_date = itinerary.get(END_DATE, "未設定")
        print(f"--- ツール: 旅行日程 開始日: {start_date}, 終了日: {end_date}, 現在時刻: {current_time} ---")

    # モックの旅行先データ（内部では常に日本語で保存）
    mock_destination_db = {
        "ニューヨーク": {
            "Japanese": "ニューヨークはアメリカ合衆国最大の都市で、自由の女神像、セントラルパーク、タイムズスクエアなどの観光名所があります。",
            "English": "New York is the largest city in the United States, with attractions such as the Statue of Liberty, Central Park, and Times Square."
        },
        "ロンドン": {
            "Japanese": "ロンドンはイギリスの首都で、バッキンガム宮殿、ビッグベン、ロンドン塔などの歴史的建造物が有名です。",
            "English": "London is the capital of the United Kingdom, famous for its historical buildings such as Buckingham Palace, Big Ben, and the Tower of London."
        },
        "東京": {
            "Japanese": "東京は日本の首都で、伝統と現代が融合した都市です。浅草寺、東京スカイツリー、渋谷などが人気の観光スポットです。",
            "English": "Tokyo is the capital of Japan, a city where tradition and modernity merge. Popular tourist spots include Sensoji Temple, Tokyo Skytree, and Shibuya."
        },
    }

    if destination in mock_destination_db:
        data = mock_destination_db[destination]

        if preferred_language in data:
            info = data[preferred_language]
        else:
            info = data["Japanese"]  # デフォルトは日本語

        # 旅行日程が設定されている場合は、情報に追加
        if itinerary and START_DATE in itinerary and END_DATE in itinerary:
            trip_info = f"\n\n旅行日程: {itinerary[START_DATE]} から {itinerary[END_DATE]} まで"
            info += trip_info

        result = {"status": "success", "info": info}
        print(f"--- ツール: {preferred_language}でレポートを生成しました。結果: {result} ---")
        return result
    else:
        # 旅行先が見つからない場合の処理
        error_msg = f"申し訳ありませんが、'{destination}'の情報はありません。"
        print(f"--- ツール: 旅行先 '{destination}' が見つかりませんでした。 ---")
        return {"status": "error", "error_message": error_msg}

print("✅ 状態認識ツール 'get_destination_info_stateful' が定義されました。")

In [ ]:
# ユーザーの好みを記憶する

def memorize(key: str, value: str, tool_context: ToolContext) -> dict:
    """
    情報を記憶します。キーと値のペアを一度に1つずつ保存します。

    引数:
        key (str): 値を保存するためのメモリのインデックスとなるラベル。
        value (str): 保存する情報。
        tool_context (ToolContext): ADKツールコンテキスト。

    戻り値:
        dict: ステータスメッセージを含む辞書。
    """
    print(f"--- ツール: memorize が呼び出されました。キー: {key}, 値: {value} ---")
    mem_dict = tool_context.state
    mem_dict[key] = value
    return {"status": f'保存しました "{key}": "{value}"'}

def memorize_list(key: str, value: str, tool_context: ToolContext) -> dict:
    """
    情報をリストとして記憶します。

    引数:
        key (str): 値を保存するためのメモリのインデックスとなるラベル。
        value (str): 保存する情報。
        tool_context (ToolContext): ADKツールコンテキスト。

    戻り値:
        dict: ステータスメッセージを含む辞書。
    """
    print(f"--- ツール: memorize_list が呼び出されました。キー: {key}, 値: {value} ---")
    mem_dict = tool_context.state
    if key not in mem_dict:
        mem_dict[key] = []
    if value not in mem_dict[key]:
        mem_dict[key].append(value)
    return {"status": f'リストに保存しました "{key}": "{value}"'}

def set_user_preference(preference_type: str, value: str, tool_context: ToolContext) -> dict:
    """ユーザーの好みを設定します。

    引数:
        preference_type (str): 好みのタイプ（例: "language", "currency", "temperature_unit"）。
        value (str): 好みの値。
        tool_context (ToolContext): セッション状態へのアクセスを提供するADKツールコンテキスト。

    戻り値:
        dict: 処理の確認またはエラーを報告する辞書。
    """
    print(f"--- ツール: set_user_preference が呼び出されました。タイプ: {preference_type}, 値: {value} ---")

    # ユーザー設定を取得または初期化
    if USER_PREFERENCES not in tool_context.state:
        tool_context.state[USER_PREFERENCES] = {}

    # 好みを設定
    tool_context.state[USER_PREFERENCES][preference_type] = value
    print(f"--- ツール: ユーザー設定を更新しました '{preference_type}': {value} ---")
    return {"status": "success", "message": f"ユーザー設定 {preference_type} を {value} に設定しました。"}

def create_itinerary(destination: str, start_date: str, end_date: str, tool_context: ToolContext) -> dict:
    """旅行の日程を作成します。

    引数:
        destination (str): 旅行先。
        start_date (str): 旅行開始日（YYYY-MM-DD形式）。
        end_date (str): 旅行終了日（YYYY-MM-DD形式）。
        tool_context (ToolContext): セッション状態へのアクセスを提供するADKツールコンテキスト。

    戻り値:
        dict: 処理の確認またはエラーを報告する辞書。
    """
    print(f"--- ツール: create_itinerary が呼び出されました。旅行先: {destination}, 開始日: {start_date}, 終了日: {end_date} ---")

    # 旅行日程を作成
    itinerary = {
        "destination": destination,
        START_DATE: start_date,
        END_DATE: end_date,
        "activities": []
    }

    # 旅行日程を保存
    tool_context.state[ITIN_KEY] = itinerary
    tool_context.state[ITIN_START_DATE] = start_date
    tool_context.state[ITIN_END_DATE] = end_date
    tool_context.state[ITIN_DATETIME] = str(datetime.now())
    tool_context.state[SYSTEM_TIME] = str(datetime.now())
    tool_context.state[ITIN_INITIALIZED] = True

    print(f"--- ツール: 旅行日程を作成しました。旅行先: {destination}, 開始日: {start_date}, 終了日: {end_date} ---")
    return {
        "status": "success", 
        "message": f"{destination}への旅行日程が作成されました。期間: {start_date} から {end_date} まで"
    }

def _initialize_session_state(callback_context: CallbackContext):
    """
    セッション状態を初期化します。
    ルートエージェントのbefore_agent_callbackとして設定します。
    システム指示が構築される前に呼び出されます。

    引数:
        callback_context: コールバックコンテキスト。
    """
    # 現在時刻を設定
    if SYSTEM_TIME not in callback_context.state:
        callback_context.state[SYSTEM_TIME] = str(datetime.now())
        print(f"--- コールバック: システム時刻を設定しました: {callback_context.state[SYSTEM_TIME]} ---")

    # 旅行日程が初期化されていない場合は初期化フラグを設定
    if ITIN_INITIALIZED not in callback_context.state:
        callback_context.state[ITIN_INITIALIZED] = False
        print("--- コールバック: 旅行日程初期化フラグを設定しました ---")

    # ユーザー設定が初期化されていない場合は初期化
    if USER_PREFERENCES not in callback_context.state:
        callback_context.state[USER_PREFERENCES] = {"language": "Japanese"}
        print("--- コールバック: ユーザー設定を初期化しました ---")

### 2\.  ルートエージェントを更新する

In [ ]:
# サブエージェントは既にインポート済み
print(f"✅ booking_agent: {booking_agent.name}")
print(f"✅ inspiration_agent: {inspiration_agent.name}")
print(f"✅ planning_agent: {planning_agent.name}")


# ルートエージェントを作成する前に前提条件を確認
if booking_agent and inspiration_agent and planning_agent and 'get_destination_info_stateful' in globals():
    root_agent_stateful = Agent(
        name="travel_agent_v3_stateful", # 新しいバージョン名
        model=DEFAULT_MODEL,
        description="メインエージェント: 旅行情報を提供し（状態認識ユニット）、予約、インスピレーション、プランニングを委任し、旅行日程を管理します。",
        instruction=f"""あなたはメインの旅行エージェントです。あなたの仕事は 'get_destination_info_stateful' を使って旅行情報を提供することです。
                    このツールは、状態に保存されているユーザーの好みに基づいて言語の形式を設定します。

                    旅行のインスピレーションが必要な場合は 'inspiration_agent' に、
                    旅行の計画が必要な場合は 'planning_agent' に、
                    予約の処理が必要な場合は 'booking_agent' に委任してください。

                    セッション状態には以下の情報が保存されています：
                    - ユーザー設定: {USER_PREFERENCES}
                    - 旅行日程: {ITIN_KEY}
                    - 現在時刻: {SYSTEM_TIME}

                    旅行日程が設定されている場合は、その情報を考慮して回答してください。
                    旅行日程が設定されていない場合は、ユーザーに旅行日程の設定を促してください。

                    旅行に関するリクエストのみを処理してください。""",
        tools=[
            get_destination_info_stateful, 
            set_user_preference,
            memorize,
            memorize_list,
            create_itinerary
        ], # 状態認識ツールを使用
        sub_agents=[booking_agent, inspiration_agent, planning_agent], # サブエージェントを含める
        output_key="last_travel_report", # エージェントの最終的な旅行応答を自動保存
        before_agent_callback=_initialize_session_state # セッション状態を初期化するコールバック
    )
    print(f"✅ ルートエージェント '{root_agent_stateful.name}' がステートフルツールと output_key を使用して作成されました。")
    print(f"✅ サブエージェント: {[sa.name for sa in root_agent_stateful.sub_agents]}")
    print(f"✅ ツール: {[tool.name for tool in root_agent_stateful.tools]}")

else:
    print("❌ ステートフルルートエージェントを作成できません。前提条件がありません。")
    if not booking_agent: print(" - booking_agent の定義がありません。")
    if not inspiration_agent: print(" - inspiration_agent の定義がありません。")
    if not planning_agent: print(" - planning_agent の定義がありません。")
    if 'get_destination_info_stateful' not in globals(): print(" - get_destination_info_stateful ツールがありません。")

### 3\. テストしてみましょう！

In [ ]:
# 初期状態を設定（before_agent_callbackで自動的に初期化されるため、最小限の設定でOK）
initial_state = {
    USER_PREFERENCES: {"language": "Japanese"}
}

client = LocalAgent(root_agent_stateful, debug=True, initial_state=initial_state)

print("------------------message (1)------------------")
_ = await client.stream("ロンドンについて教えてください")

print("------------------message (2)------------------")
_ = await client.stream("言語設定を英語に変更してください")

print("------------------message (3)------------------")
_ = await client.stream("ニューヨークについて教えてください")

print("------------------message (4)------------------")
_ = await client.stream("2024年12月25日から2024年12月30日まで東京への旅行を計画しています")

print("------------------message (5)------------------")
_ = await client.stream("東京について教えてください")

# セッション状態の内容を確認
print("\n------------------セッション状態------------------")
print(f"システム時刻: {client._session.state.get(SYSTEM_TIME)}")
print(f"旅行日程初期化フラグ: {client._session.state.get(ITIN_INITIALIZED)}")
print(f"ユーザー設定: {client._session.state.get(USER_PREFERENCES)}")
print(f"旅行日程: {client._session.state.get(ITIN_KEY)}")
print(f"旅行開始日: {client._session.state.get(ITIN_START_DATE)}")
print(f"旅行終了日: {client._session.state.get(ITIN_END_DATE)}")
print(f"旅行日時: {client._session.state.get(ITIN_DATETIME)}")

## 第４章：Agent Engine にDeploy


In [ ]:
from vertexai import agent_engines
from vertexai.preview.reasoning_engines import AdkApp

In [ ]:
app = AdkApp(
    agent=travel_agent,
    enable_tracing=True,
)

In [ ]:
remote_agent = agent_engines.create(
    app,
    requirements=[
            "google-adk (==1.2.1)",
            "google-cloud-aiplatform[agent_engines] (==1.97.0)",
            "google-genai==1.20.0"
    ],
    display_name="Travel Agent 1.0",
    description="Agent Engine workshop sample",
)

In [ ]:
class RemoteApp:
    def __init__(self, remote_agent, user_id="default_user"):
        self._remote_agent = remote_agent
        self._user_id = user_id
        self._session = remote_agent.create_session(user_id=self._user_id)

    def _stream(self, query):
        events = self._remote_agent.stream_query(
            user_id=self._user_id,
            session_id=self._session['id'],
            message=query,
        )
        result = []
        for event in events:
            if ('content' in event and 'parts' in event['content']):
                response = '\n'.join(
                    [p['text'] for p in event['content']['parts'] if 'text' in p]
                )
                if response:
                    print(response)
                    result.append(response)
        return result

    def stream(self, query):
        # Retry 4 times in case of resource exhaustion 
        for c in range(4):
            if c > 0:
                time.sleep(2**(c-1))
            result = self._stream(query)
            if result:
                return result
            if DEBUG:
                print('----\nRetrying...\n----')
        return None # Permanent error

In [ ]:
remote_client = RemoteApp(remote_agent)
_ = remote_client.stream('東京について教えてください')

### 補足

#### ADK web の UI を使用する場合

GUI のチャット画面（ADK web）を試したい場合は、Cloud Workstationから次の手順で試す事ができます。

※ あくまでお試し用の手順なので、ADK web のすべての機能は使用できません。簡易的な動作確認として利用してください。

1. 作業用ディレクトリ `workdir` を作成して、カレントディレクトリに変更します。

```
mkdir workdir
cd workdir
```

2. `google-adk` のパッケージをインストールします。

```
python -m venv .venv
source .venv/bin/activate
pip install google-adk==1.2.1
```

3. リモートエージェントに接続するコードを用意します。

```
mkdir agent
cat <<EOF >agent/agent.py
import os
from uuid import uuid4
from dotenv import load_dotenv
from google.adk.agents.callback_context import CallbackContext
from google.adk.models import LlmResponse, LlmRequest
from google.adk.agents.llm_agent import LlmAgent
from google.genai.types import Content, Part

import vertexai
from vertexai import agent_engines

load_dotenv('.env')
PROJECT_ID = os.environ['PROJECT_ID']
AGENT_ID = os.environ['AGENT_ID']
LOCATION = 'us-central1'

vertexai.init(project=PROJECT_ID, location=LOCATION)
remote_agent = agent_engines.get(AGENT_ID)

async def call_remote_agent(
    callback_context: CallbackContext, llm_request: LlmRequest
) -> LlmResponse:
    session = remote_agent.create_session(user_id='default_user')
    events = remote_agent.stream_query(
                user_id='default_user',
                session_id=session['id'],
                message=str(llm_request.contents)
             )
    content = list(events)[-1]['content']
    remote_agent.delete_session(
        user_id='default_user',
        session_id=session['id'],
    )
    return LlmResponse(content=content)

root_agent = LlmAgent(
    name='remote_agent_proxy',
    model='gemini-2.0-flash', # not used
    description='Interactive agent',
    before_model_callback=call_remote_agent,
)
EOF
```

4. 設定ファイル `agent/.env` を次の内容で作成します。（`your project ID` と `your agent ID` は実際のプロジェクト ID と先ほど確認したエージェントの ID を記入します。）

```
PROJECT_ID="your project ID"
AGENT_ID="your agent ID"
```

5. チャットアプリ（ADK web）を起動します。

```
adk web
```

6. Cloud Shell の「Web でプレビュー」ボタンからポート 8000 に接続して使用します。

#### トレースの確認
Cloud Console の https://console.cloud.google.com/traces/list Trace Explorer から Agent Engine 上で実行されたエージェントのトレースが確認できます。

In [ ]:
for remote_agent in agent_engines.list():
  print(remote_agent.resource_name)

### 後片付け
デプロイしたエージェントを削除します。

In [ ]:
# 削除
for remote_agent in agent_engines.list():
  remote_agent.delete(force=True)
